In [1]:
# Import module
import pymysql
import pandas as pd

# Define DB Executor
class DatabaseExecutor(object):
    
    def __init__(self, **connection_kwargs):
        self.db = pymysql.connect(**connection_kwargs)
        self.cursor = self.db.cursor(pymysql.cursors.DictCursor)
        self.rst = None
    
    def __del__(self):
        if self.cursor.connection is not None:
            self.cursor.close()
        if not self.db._closed:
            self.db.close()
    
    def __call__(self, query, raise_exception=False):
        try:
            self.cursor.execute(query)
            self.rst = self.cursor.fetchall()
            if self.rst:
                return pd.DataFrame(self.rst)
        except Exception as e:
            if raise_exception:
                raise
            print("{0}: {1}".format(e.__class__.__name__, e))
    
    def recent(self):
        return self.rst
    

# Initialize variable
q = DatabaseExecutor(charset='utf8', unix_socket="/var/lib/mysql/mysql.sock")


In [2]:
q("use mimic")

In [3]:
q("select datediff(outtime, intime), los from mimic.icustays;")

,"datediff(outtime, intime)",los
0,2,1.6325000000
1,14,13.8507000000
2,3,2.6499000000
3,2,2.1436000000
4,2,1.2938000000
...,...,...
131,3,3.6174000000
132,1,0.6259000000
133,31,31.1235000000
134,1,1.3279000000


In [4]:
q("select TIMESTAMPDIFF(second, intime, outtime)/(60*60*24), los from mimic.icustays")

,"TIMESTAMPDIFF(second, intime, outtime)/(60*60*24)",los
0,1.6325,1.6325000000
1,13.8507,13.8507000000
2,2.6499,2.6499000000
3,2.1436,2.1436000000
4,1.2938,1.2938000000
...,...,...
131,3.6174,3.6174000000
132,0.6259,0.6259000000
133,31.1235,31.1235000000
134,1.3279,1.3279000000


In [5]:
q("select datediff('2021-03-21 23:59:59', '2021-03-21 00:00:01')")

,"datediff('2021-03-21 23:59:59', '2021-03-21 00:00:01')"
0,0


In [6]:
q("select datediff('2021-03-22 00:00:01', '2021-03-21 23:59:59')")

,"datediff('2021-03-22 00:00:01', '2021-03-21 23:59:59')"
0,1


In [7]:
q("""select TIMESTAMPDIFF(year, '1970-03-22', '2021-03-21')
+ ((month('1970-03-22')<=month('2021-03-21')) + (day('1970-03-22')<=day('2021-03-21'))>=2)
as sample_age;""")

,sample_age
0,50


In [8]:
q("""select TIMESTAMPDIFF(year, DOB ,DOD)
+ ((month(DOB)<=month(DOD)) + (day(DOB)<=day(DOD))>=2)
as age_at_death from mimic.patients
;""")

,age_at_death
0,72
1,37
2,88
3,76
4,48
...,...
95,56
96,301
97,45
98,75


In [9]:
q("""DROP PROCEDURE IF EXISTS calc_age_at_death;""")

InternalError: (1370, "alter routine command denied to user 'MED53'@'localhost' for routine 'mimic.calc_age_at_death'")


In [10]:
q("""CREATE PROCEDURE calc_age_at_death()
    BEGIN
        select TIMESTAMPDIFF(year, DOB, DOD)
        +((month(DOB)<=month(DOD))+(day(DOB)<=day(DOD))>=2) as age_at_death from patients;
    END;
""")

OperationalError: (1044, "Access denied for user 'MED53'@'localhost' to database 'mimic'")


In [11]:
q("""call calc_age_at_death();""")

InternalError: (1370, "execute command denied to user 'MED53'@'localhost' for routine 'mimic.calc_age_at_death'")


In [12]:
q("""DROP FUNCTION IF EXISTS calc_age;""")

InternalError: (1370, "alter routine command denied to user 'MED53'@'localhost' for routine 'mimic.calc_age'")


In [13]:
q("""CREATE FUNCTION calc_age(date_1 datetime, date_2 datetime)
    returns INT
    BEGIN
        return TIMESTAMPDIFF(year, date_1, date_2)+
        ((month(date_1)<=month(date_2))+(day(date_1)<=day(date_2))>=2);
    END
;""")

OperationalError: (1044, "Access denied for user 'MED53'@'localhost' to database 'mimic'")


In [14]:
q("select calc_age(DOB, DOD) as age_at_death from patients;")

InternalError: (1370, "execute command denied to user 'MED53'@'localhost' for routine 'mimic.calc_age'")


In [15]:
q("USE med2131")

In [16]:
q("show tables")

,Tables_in_med2131
0,subjects_0114349
1,subjects_0114349_2
2,subjects_114340
3,subjects_114349
4,subjects_114349_2
...,...
72,subjects_2020191119
73,subjects_2020191121
74,subjects_2020191121_2
75,view_0114349


In [17]:
q("""DROP TABLE dash_personal""")

In [ ]:
q("""DROP TABLE dash_personal_2""")

In [19]:
q("""CREATE TABLE dash_personal
(subject_id int, admtime datetime, age_at_adm int, age_at_death int, 
primary key (subject_id));
""")

In [20]:
q("""show columns from dash_personal""")

,Field,Type,Null,Key,Default,Extra
0,subject_id,int(11),NO,PRI,None,
1,admtime,datetime,YES,,None,
2,age_at_adm,int(11),YES,,None,
3,age_at_death,int(11),YES,,None,


In [21]:
q("alter table dash_personal add death_or_not varchar(1);")

In [22]:
q("""show columns from dash_personal""")

,Field,Type,Null,Key,Default,Extra
0,subject_id,int(11),NO,PRI,None,
1,admtime,datetime,YES,,None,
2,age_at_adm,int(11),YES,,None,
3,age_at_death,int(11),YES,,None,
4,death_or_not,varchar(1),YES,,None,


In [23]:
q("alter table dash_personal modify column death_or_not int;")

In [24]:
q("alter table dash_personal drop column death_or_not")

In [25]:
q("""insert into dash_personal
    select distinct A.subject_id, B.admittime, TIMESTAMPDIFF(year, A.DOB, B.ADMITTIME),
        TIMESTAMPDIFF(year, A.DOB, B.DEATHTIME)
    from mimic.patients as A, mimic.admissions as B where A.subject_id=B.subject_id limit 10""")

In [26]:
q("""select * from dash_personal""")

,subject_id,admtime,age_at_adm,age_at_death
0,10006,2164-10-23 21:09:00,70,NaN
1,10011,2126-08-14 22:32:00,36,36.0
2,10013,2125-10-04 23:36:00,87,87.0
3,10017,2149-05-26 17:19:00,73,NaN
4,10019,2163-05-14 20:43:00,48,48.0
5,10026,2195-05-17 07:39:00,300,NaN
6,10027,2190-07-13 07:15:00,82,NaN
7,10029,2139-09-22 10:58:00,78,NaN
8,10032,2138-04-02 19:52:00,88,NaN
9,10033,2132-12-05 02:46:00,81,NaN


In [27]:
q("""create table dash_personal_2 as
    select distinct A.subject_id, B.admittime, TIMESTAMPDIFF(year, A.DOB, B.ADMITTIME) as age_at_adm,
        TIMESTAMPDIFF(year, A.DOB, B.DEATHTIME) as age_at_death
    from mimic.patients as A, mimic.admissions as B where A.subject_id=B.subject_id limit 10""")

In [28]:
q("show columns from dash_personal_2")

,Field,Type,Null,Key,Default,Extra
0,subject_id,mediumint(8) unsigned,NO,,None,
1,admittime,datetime,NO,,None,
2,age_at_adm,bigint(21),YES,,None,
3,age_at_death,bigint(21),YES,,None,


In [29]:
q("""select * from dash_personal_2""")

,subject_id,admittime,age_at_adm,age_at_death
0,10006,2164-10-23 21:09:00,70,NaN
1,10011,2126-08-14 22:32:00,36,36.0
2,10013,2125-10-04 23:36:00,87,87.0
3,10017,2149-05-26 17:19:00,73,NaN
4,10019,2163-05-14 20:43:00,48,48.0
5,10026,2195-05-17 07:39:00,300,NaN
6,10027,2190-07-13 07:15:00,82,NaN
7,10029,2139-09-22 10:58:00,78,NaN
8,10032,2138-04-02 19:52:00,88,NaN
9,10033,2132-12-05 02:46:00,81,NaN


In [30]:
q("update dash_personal set age_at_death=72 where subject_id=10006")

In [31]:
q("delete from dash_personal where age_at_death is null")
# q("delete from dash_personal where isnull(age_at_death)=1")

In [32]:
q("""select * from dash_personal""")

,subject_id,admtime,age_at_adm,age_at_death
0,10006,2164-10-23 21:09:00,70,72
1,10011,2126-08-14 22:32:00,36,36
2,10013,2125-10-04 23:36:00,87,87
3,10019,2163-05-14 20:43:00,48,48
